At UCHT we drive Notes from Nature expeditions for multiple TN collections. This script makes it easier to update each of those collections in SERNEC given the mixed institution results returned from a completed NfN expedition. 

Essentially this script merges all expedition results into a single dataframe, performs basic cleaning then splits on collection type.

Process is simple:
 - unzip all expeditions and drop them in the root of this folder 
     - each expedition results as their own subfolder
 - run this script
 - upload the individual collection's csvs using SERNEC's "Home >> Collection Management Panel >> Import/Update Specimen Records  >> Notes from Nature Import"

In [1]:
import pandas as pd
from glob import glob
import re

In [2]:
# gather all reconcilations, align older expedition col names with new ones.
records = glob('*/**/*reconcile*.csv', recursive=True)
df = pd.DataFrame()
for csv in records:
    df = df.append(pd.read_csv(csv), ignore_index=True, sort=True)
# align the old and new column names
df['Habitat & Description'] = df['Habitat & Description'].fillna(df['Habitat_Description'])
# organize a list of cols to remove
# Habitat_Description is not redundant
# subject_scientificName was not transcribed in our NFN expeditions, & we don't want to override new annotations.
dropCols = ['Habitat_Description', 'subject_scientificName']

# duplicate habitat and description (which maps to verbatumattributes)
df['habitat'] = df['Habitat & Description']
df['description'] = df['Habitat & Description']

# remove those "Not shown" entries
dropTerms = ['#','not shown','s.n.n.','s.n.n','s.n.','s.n',
             'N/a','No Number','none']

for dateCol in ['Day','Month','Year']:
    for dropTerm in dropTerms:
        df[dateCol] = df[dateCol].str.replace(dropTerm, '',flags=re.IGNORECASE)
    df[dateCol] = df[dateCol].str.strip()

# align old names
for oldName in ['Collected_By','Collector_Number']:
    newName = oldName.replace('_',' ')
    df[newName] = df[newName].fillna(df[oldName])
    dropCols.append(oldName)
    
# clean up a few common extraneous characters in 'Collector Number'
for dropTerm in dropTerms:
    df['Collector Number'] = df['Collector Number'].str.replace(dropTerm, '', flags=re.IGNORECASE)

df['Collector Number'] = df['Collector Number'].str.rstrip('.').str.strip()

for dropCol in dropCols:
    df.drop(dropCol, axis=1, inplace=True)

# see what we got
display(df.shape)
display(df.columns)
display(df.sample(3))

(17257, 26)

Index(['Collected By', 'Collector Number', 'Day', 'Habitat & Description',
       'Location', 'Month', 'Year', 'subject_catalogNumber',
       'subject_collectionCode', 'subject_country', 'subject_county',
       'subject_eol', 'subject_expeditionId', 'subject_expeditionTitle',
       'subject_id', 'subject_idigbio', 'subject_imageName',
       'subject_imageURL', 'subject_institutionCode', 'subject_mol',
       'subject_recordId', 'subject_references', 'subject_stateProvince',
       'subject_subjectId', 'habitat', 'description'],
      dtype='object')

,Collected By,Collector Number,Day,Habitat & Description,Location,Month,Year,subject_catalogNumber,subject_collectionCode,subject_country,...,subject_imageName,subject_imageURL,subject_institutionCode,subject_mol,subject_recordId,subject_references,subject_stateProvince,subject_subjectId,habitat,description
3226,Julia Ann Sparkman,34,22,waste places,"near Doyle, TN",7 - July,1963,HTTU016271,NaN,United States,...,5a84b85bae5bfd4011251dd3.jpg,NaN,HTTU,https://www.mol.org/species/Ailanthus%20glandu...,urn:uuid:40132639-fec0-4478-822f-0ac06bbfafe2,http://sernecportal.org/portal/collections/ind...,Tennessee,5a84b85bae5bfd4011251dd3,waste places,waste places
9477,S. A. Cain,NaN,29,Pine woods (dry),East of Chilhowee Mt. Montvale.,6 - June,1930,TENN-V-0128437,NaN,United States,...,5ad7690cae5bfd312d3080dc.jpg,https://bisque.cyverse.org/image_service/image...,TENN,https://www.mol.org/species/Stylosanthes%20bif...,urn:uuid:4ce0d692-e71e-4cdb-b3aa-904ea7683d8b,http://sernecportal.org/portal/collections/ind...,Tennessee,5ad7690cae5bfd312d3080dc,Pine woods (dry),Pine woods (dry)
7191,R. Dale Thomas,389,4,Pine woods. Elev. 2750,Chilhowee Mountain\nRoadbank near Millstone Ga...,10 - October,1965,TENN-V-0035025,NaN,United States,...,5ad76903ae5bfd312d307c56.jpg,https://bisque.cyverse.org/image_service/image...,TENN,https://www.mol.org/species/Bromus%20japonicus,urn:uuid:a00a128f-35f6-468d-9e60-f72c75d351f7,http://sernecportal.org/portal/collections/ind...,Tennessee,5ad76903ae5bfd312d307c56,Pine woods. Elev. 2750,Pine woods. Elev. 2750


In [16]:
unique_codes = df['subject_institutionCode'].unique()
for code in unique_codes:
    tempDF = df.loc[df['subject_institutionCode'] == code]
    if len(tempDF) > 0: # be sure there is something beyond NaN
        tempDF.fillna('').to_csv(f'{code}_combined_nfn.csv', index=False)